<h2>Lesson subject: introducing parallelism</h2>

In [ ]:
#load """..\src\Domain.fsx""" 
#load """..\src\Cities\Toronto.fsx""" 

open Domain

Toronto.license.info |> display 

let addresses , districts, streets, summary = 
    setup Toronto.data Area.Neighbourhoods (Limit.Streets 500) 

summary |> display

let positions = addresses |> Array.map (fun x -> x.Pos) |> Array.toList

let indexedDistricts = districts |> Array.indexed

Open Government Licence – Toronto
(https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-licence/)
Attribution: Contains information licensed under the Open Government Licence – Toronto

data summary (without limits) -> 
 areas: 140, streets: 9521, addressess: 527189 
 addresses per street (median/mean/std-dev): 32.0/55.3711795/104.1107272

In [ ]:
let hintSearch lastHitAreaIndex currentPosition  =
    let hintArea = indexedDistricts.[lastHitAreaIndex]
    let find = snd >> isPointWithinFeature currentPosition

    if find hintArea then lastHitAreaIndex
    else indexedDistricts |> Array.findIndex find

let results = 
    streets |> Array.Parallel.map (fun street -> 
        scan hintSearch 0 street.Positions
    ) |> Array.collect id 

Remember that we no longer have sequential processing so the addresses are not sorted as they use to be. In order to make them usable we have to return index as well.

In [ ]:
let hintSearch (_, lastHitAreaIndex) (currentPosition :Address) =
    let hintArea = indexedDistricts.[lastHitAreaIndex]
    let find = snd >> isPointWithinFeature currentPosition.Pos

    if find hintArea then currentPosition.Id, lastHitAreaIndex
    else currentPosition.Id, indexedDistricts |> Array.findIndex find

let results = 
    streets |> Array.Parallel.map (fun street -> 
        let x = street.Addresses |> Array.item 0
        scan hintSearch (x.Id, 0) street.Addresses  
    ) |> Array.collect id 

In [ ]:
results 
    |> Seq.skip 520 
    |> Seq.take 10 
    |> Seq.mapi (fun i x -> {| AddressIndex = i + 520; MatchedAreaIndex = x |})

index AddressIndex MatchedAreaIndex 0 520 Item1 Item2 9e280c1ebfcee67f 48 1 521 Item1 Item2 62b605c8e40688b4 48 2 522 Item1 Item2 7253d21abd885780 48 3 523 Item1 Item2 f406d457f68d4a28 48 4 524 Item1 Item2 e9da8a0139a34d82 48 5 525 Item1 Item2 6979e1a7b804eb7f 48 6 526 Item1 Item2 aa8c9fccfaee0f81 48 7 527 Item1 Item2 9dba0858c2666086 48 8 528 Item1 Item2 2584030c01f55064 48 9 529 Item1 Item2 b5ac27a97f194385 16